# Simulação da frota de taxis

$n \in \Bbb{N}$ taxis saem de casa.

Cada um faz 3 corridas e vai pra casa.

Cada taxi começa a circular, encontra um passageiro, leva o passageiro, deixa o passageiro e volta a circular.

O tempo até achar um passageiro é $t \sim Exp(\lambda)$.

Para uma exibição mais limpa, considere $\lceil t\rceil$.

Mudanças de estados de taxis são registradas como eventos.

Os taxis 0, 1, 2, ... saem de suas casas nessa ordem e com 5 minutos de diferença.

## Importações

In [41]:
import random
import queue
from collections import namedtuple
from enum import Enum
from math import ceil

## Constantes

In [42]:
DEFAULT_NUMBER_OF_TAXIS = 5
DEFAULT_END_TIME = 80
SEARCH_DURATION = 3
TRIP_DURATION = 13
DEPARTURE_INTERVAL = 5

class Acao(Enum):
    SAIU = "saiu da garagem"
    PEGOU = "pegou um passageiro"
    DEIXOU = "deixou o passageiro"
    ACABOU = "indo pra casa descansar"

## Definição de evento

In [43]:
Evento = namedtuple("Evento", "instante carro acao")

## Delegante

In [44]:
def taxi_processo(carro, qtd_corridas=3, tempo_inicial=0):
    tempo = yield Evento(tempo_inicial, carro, Acao.SAIU)
    for _ in range(qtd_corridas):
        tempo = yield Evento(tempo, carro, Acao.PEGOU)
        tempo = yield Evento(tempo, carro, Acao.DEIXOU)
    yield Evento(tempo, carro, Acao.ACABOU)

## Helper

In [45]:
def calcula_duracao(acao_anterior):
    """Calcula a duração da ação usando a distribuição exponencial"""
    if acao_anterior in [Acao.SAIU, Acao.DEIXOU]:
        interval = SEARCH_DURATION
    elif acao_anterior == Acao.PEGOU:
        interval = TRIP_DURATION
    elif acao_anterior == Acao.ACABOU:
        interval = 1
    else:
        raise ValueError(f"Valor inválido para 'acao_anterior': {acao_anterior}")
    return ceil(random.expovariate(1 / interval))

## Classe *Simulador*

In [46]:
class Simulador:
    def __init__(self, processos_list):
        self.eventos = queue.PriorityQueue()
        self.processos = processos_list

    def run(self, tempo_limite):
        for proc in self.processos:
            primeiro_evento = next(proc)
            self.eventos.put(primeiro_evento)

        instante_simulacao = 0
        while instante_simulacao < tempo_limite:
            if self.eventos.empty():
                print("*** fim dos eventos ***")
                break

            evento_atual = self.eventos.get()
            instante_simulacao, id_carro, acao_anterior = evento_atual
            print(f"taxi {id_carro}; t={instante_simulacao:3d}:", id_carro * "   ", evento_atual)
            tempo_seguinte = instante_simulacao + calcula_duracao(acao_anterior)
            try:
                if 0 <= id_carro < len(self.processos):
                    processo_ativo = self.processos[id_carro]
                    proximo_evento = processo_ativo.send(tempo_seguinte)
            except StopIteration:
                pass
            else:
                self.eventos.put(proximo_evento)
        else:
            msg = "*** Fim da execução da simulação: {} eventos pendentes ***"
            print(msg.format(self.eventos.qsize()))

## Rodando a simulação

In [47]:
def main(end_time=DEFAULT_END_TIME, num_taxis=DEFAULT_NUMBER_OF_TAXIS):
    taxis = [
        taxi_processo(i, 2, i * DEPARTURE_INTERVAL)
        for i in range(num_taxis)
    ]
    sim = Simulador(taxis)
    sim.run(end_time)


main()

taxi 0; t=  0:  Evento(instante=0, carro=0, acao=<Acao.SAIU: 'saiu da garagem'>)
taxi 0; t=  1:  Evento(instante=1, carro=0, acao=<Acao.PEGOU: 'pegou um passageiro'>)
taxi 0; t=  2:  Evento(instante=2, carro=0, acao=<Acao.DEIXOU: 'deixou o passageiro'>)
taxi 0; t=  3:  Evento(instante=3, carro=0, acao=<Acao.PEGOU: 'pegou um passageiro'>)
taxi 1; t=  5:     Evento(instante=5, carro=1, acao=<Acao.SAIU: 'saiu da garagem'>)
taxi 1; t=  6:     Evento(instante=6, carro=1, acao=<Acao.PEGOU: 'pegou um passageiro'>)
taxi 2; t= 10:        Evento(instante=10, carro=2, acao=<Acao.SAIU: 'saiu da garagem'>)
taxi 2; t= 12:        Evento(instante=12, carro=2, acao=<Acao.PEGOU: 'pegou um passageiro'>)
taxi 3; t= 15:           Evento(instante=15, carro=3, acao=<Acao.SAIU: 'saiu da garagem'>)
taxi 1; t= 20:     Evento(instante=20, carro=1, acao=<Acao.DEIXOU: 'deixou o passageiro'>)
taxi 4; t= 20:              Evento(instante=20, carro=4, acao=<Acao.SAIU: 'saiu da garagem'>)
taxi 1; t= 21:     Evento(inst

A simulação ocorre graças à ordenação automática da fila de prioridades em `Simulação.eventos`, pois isto faz com que os eventos ocorram conforme seu agendamento. Esse é o coração do algoritmo.

A corrotina poderia ser substituída por uma função com o processo no closure.